<a href="https://colab.research.google.com/github/dhighlord/NNCTR/blob/master/run_multivalue_movielens_hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dhighlord/NNCTR.git

Cloning into 'NNCTR'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 75 (delta 2), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (75/75), done.


In [0]:
import numpy as np
import pandas as pd
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from NNCTR.deepctr.models import DeepFM
from NNCTR.deepctr.inputs import SparseFeat, VarLenSparseFeat,get_fixlen_feature_names

In [5]:
data = pd.read_csv("NNCTR/examples/movielens_sample.txt")
sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zip", ]

data[sparse_features] = data[sparse_features].astype(str)
target = ['rating']

# 1.Use hashing encoding on the fly for sparse features,and process sequence features

genres_list = list(map(lambda x: x.split('|'), data['genres'].values))
genres_length = np.array(list(map(len, genres_list)))
max_len = max(genres_length)

# Notice : padding=`post`
genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', dtype=str, value=0)

# 2.set hashing space for each sparse field and generate feature config for sequence feature

fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique() * 5, use_hash=True, dtype='string')
                          for feat in sparse_features]
varlen_feature_columns = [VarLenSparseFeat('genres', 100, max_len, 'mean', use_hash=True,
                                           dtype="string")]  # Notice : value 0 is for padding for sequence input feature
linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns
feature_names = get_fixlen_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for model
fixlen_input = [data[name].values for name in feature_names]
varlen_input = [genres_list]

model_input = fixlen_input + varlen_input # make sure the order is right

# 4.Define Model,compile and train
model = DeepFM(linear_feature_columns,dnn_feature_columns, task='regression')

model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(model_input, data[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )


W0809 10:20:21.148170 140396070111104 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0809 10:20:21.363442 140396070111104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/converters/directives.py:117: The name tf.string_to_hash_bucket_fast is deprecated. Please use tf.strings.to_hash_bucket_fast instead.

W0809 10:20:22.241670 140396070111104 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py:253: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated

Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 - 1s - loss: 14.3009 - mean_squared_error: 14.3009 - val_loss: 13.3791 - val_mean_squared_error: 13.3791
Epoch 2/10
160/160 - 0s - loss: 14.1576 - mean_squared_error: 14.1576 - val_loss: 13.2492 - val_mean_squared_error: 13.2492
Epoch 3/10
160/160 - 0s - loss: 14.0041 - mean_squared_error: 14.0041 - val_loss: 13.1089 - val_mean_squared_error: 13.1089
Epoch 4/10
160/160 - 0s - loss: 13.8389 - mean_squared_error: 13.8389 - val_loss: 12.9594 - val_mean_squared_error: 12.9594
Epoch 5/10
160/160 - 0s - loss: 13.6629 - mean_squared_error: 13.6629 - val_loss: 12.8001 - val_mean_squared_error: 12.8001
Epoch 6/10
160/160 - 0s - loss: 13.4755 - mean_squared_error: 13.4755 - val_loss: 12.6301 - val_mean_squared_error: 12.6301
Epoch 7/10
160/160 - 0s - loss: 13.2757 - mean_squared_error: 13.2757 - val_loss: 12.4482 - val_mean_squared_error: 12.4482
Epoch 8/10
160/160 - 0s - loss: 13.0618 - mean_squared_error: 13.0618 - val_loss: 12.25